In [3]:
from matplotlib import pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np

import sys

sys.path.insert(0, "../pompjax/pompjax/")

from ifeakf import ifeakf
from utils.samples2df import create_df_response
from diagnostic_plots.diagnostic_plot import convergence_plot

sys.path.insert(0, "../")

from models import process_metapop, observe_metapop, init_metapop, simulate

from global_config import config
import os

results_dir = config.get_property('results_dir')
model_name  = "mp model mean streams" #
#results_dir = os.path.join("..", "identifiability", model_name)
results_dir = os.path.join(results_dir, "identifiability", model_name)

os.makedirs(results_dir, exist_ok=True)

plt.rc('axes', labelsize=12)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', titlesize=8)


In [4]:
#adht_building = pd.read_csv('../../data/admissions_discharges_building.csv', parse_dates=['date'])
adht_building = pd.read_csv('../../data/counts_building.csv', parse_dates=['date'])

date_start = pd.to_datetime('2020-02-03')
date_end   = pd.to_datetime('2021-02-28')
dates      = pd.date_range(date_start, date_end)
Tdat       = len(dates)

date_start_infer = pd.to_datetime('2020-03-01')
date_end_infer   = pd.to_datetime('2021-02-28')
dates_infer      = pd.date_range(date_start_infer, date_end_infer)
idx_dates_infer  = np.nonzero(np.in1d(dates,dates_infer))[0]

#adht_building.date = adht_building.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
adht_building = adht_building[adht_building.date>=date_start]
adht_building = adht_building[adht_building.date<=date_end]

#selected_building = ['Allen Hospital', 'Harkness Pavilion', 'Herbert Irving Pavilion', 'Milstein Family Heart Ctr', 'Milstein Hospital', 'Mschony', 'Presbyterian Hospital']
selected_building = ['Allen Hospital', 'Harkness Pavilion', 'Milstein Hospital', 'Mschony', 'Presbyterian Hospital']
adht_building     = adht_building[adht_building.building.isin(selected_building)]

A_df     = pd.pivot(adht_building,index='building',columns='date',values='num_admitted')
D_df     = pd.pivot(adht_building,index='building',columns='date',values='num_discharged')
H_df     = pd.pivot(adht_building,index='building',columns='date',values='num_hospitalized')
tests_df = pd.pivot(adht_building,index='building',columns='date',values='num_tested')

pop            = H_df.mean(axis=1)
num_pop        = len(pop)
building_names = pop.index

building_num = len(building_names)
building_transfers = pd.read_csv('../../data/transfers_building.csv', parse_dates=['date'])
building_transfers = building_transfers[building_transfers.date>=date_start]
building_transfers = building_transfers[building_transfers.date<=date_end]

M_df = np.zeros((building_num,building_num,Tdat+1))
for i in range(0,building_num):
    building_from = building_names[i]
    for j in range(0,building_num):
        building_to = building_names[j]
        transfers_ij = building_transfers[(building_transfers.building_from==building_from) & (building_transfers.building_to==building_to)]
        if(transfers_ij.shape[0] > 0) :
            dates_ij = transfers_ij.date.values
            dates_ind = np.where(np.in1d(dates_ij, dates))[0]
            transfered = transfers_ij.num_transfered.values
            #M_df[i,j,dates_ind] = transfered
            M_df[i,j,dates_ind-1] = transfered

FileNotFoundError: [Errno 2] No such file or directory: '../../data/counts_building.csv'